In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException, NoSuchElementException
import time
import re
import json
from wakepy import keepawake
from scraper_utils import *

C:\Users\gkjac\AppData\Local\Temp\ipykernel_10584\3292858552.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
driver = webdriver.Chrome()
driver.maximize_window()

# Open Instacart to your desired store
driver.get(f'https://www.instacart.com/')

In [3]:
# scrape nearby store names and their instacart site urls from Instacart's main page
store_dict = {}
store_icons = driver.find_element(By.XPATH, "//main//ul")
for li in store_icons.find_elements(By.XPATH, ".//li"):
    store = li.find_element(By.CSS_SELECTOR, 'div:nth-child(1) div:nth-child(1) a')

    # find store name stored as text within icon
    store_name = store.find_element(By.CSS_SELECTOR, ' a span').text

    # find url that clicking on icon would take you too
    url = store.get_attribute('href')
    
    store_dict.update({store_name:url})
    
driver.quit()


In [4]:
for store in store_dict:
    print(store)

ALDI
ALDI Express
Costco
Dave's Markets
Giant Eagle
Target
Target: Fast Delivery
Fresh Thyme Market
Walgreens
CVS®
BJ's Wholesale Club
Grove Kosher Market


In [22]:
def carousel_filter(carousels):
    mapping = []
    final_carousels = []
    
    for x in range(len(carousels)):
        
        try:
            
            #category_label = carousels[x].find_element(By.CSS_SELECTOR, 'div:nth-child(1) div:nth-child(1) h2 '
                                                                        #'div:nth-child(1)')
            category_label = carousels[x].find_element(By.CSS_SELECTOR, 'h2')
            
        except NoSuchElementException:
            continue
        category_text = category_label.text

        mapping.append(category_text)

        final_carousels.append(carousels[x])


    return final_carousels, mapping

In [23]:
driver = webdriver.Chrome()
driver.maximize_window()
for store in store_dict:
    # Define your store name. original_store is used in the dataframe
    # and file name creation if the store name has spaces
    #store = "Giant Eagle"
    original_store = store
    url = store_dict.get(store)
    # If the store has special characters like spaces and apostrophes, replace them with dashes to make them compatible
    # with instacart URLs
    #if ' ' in store:
        #store = '-'.join(store.split())
    store = re.sub("[^A-Za-z]","",store)

    # Initialize the df to store all of our data
    df = pd.DataFrame(columns=["Store", "Item Name", "Price", "Units", "Unit Price", "Amount", "Tags", "In Stock", "Image"])
    
    # Load your webdriver
    # driver = webdriver.Chrome(ChromeDriverManager().install())
    
    # Open Instacart to your desired store
    #driver.get(f'https://www.instacart.com/{store}')
    driver.get(url)
    wait = WebDriverWait(driver, 5)  # Maximum wait time of 5 seconds

    # Remove modals that block website
    elements_to_delete = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ReactModalPortal')))
    
    for element in elements_to_delete:
        driver.execute_script("arguments[0].remove();", element)
    
    try:
        express_popup = driver.find_element(By.CSS_SELECTOR, 'div[aria-label="Need a few things fast?"] div:nth-child(3) button')
        express_popup.click()
    except NoSuchElementException:
        continue

    # This allows us to scroll and click again
    body_element = driver.find_element(By.TAG_NAME,'body')
    driver.execute_script("arguments[0].style.overflow = 'visible';", body_element)

    # Scroll slowly to the bottom
    slow_scroll_to_bottom(driver)

    # Wait 2 seconds to allow for final loading
    time.sleep(2)

    # Find all elements with aria-label="item carousel"
    # These are the product groupings we want to jump into
    carousels = driver.find_elements(By.CSS_SELECTOR, '[aria-label="item carousel"]')
    print(carousels)
    carousels, mapping = carousel_filter(carousels)
    print('carousels:')
    print(carousels)
    # These divs contain the buttons we need to click on within each item carousel
    portals = driver.find_elements(By.CLASS_NAME, '__reakit-portal')

    # Some reakit portals do not correspond to carousels
    fake_labels = ['Welcome', 'Join an Instacart+ family account dialog', 'Cart', 'Coupon',
                'Use backup payment method for added items?', '100% satisfaction guarantee', '']

    reakit_portals = reakit_filter(portals, fake_labels, mapping)

    # List of dataframes to be concatenated later.
    to_concat = []

    # If we remove the useless portals, we can start from 0
    j = 0

    # Iterate through each div
    for carousel in carousels[0:2]:
        print(j)
        # Scroll to where the carousel is located and click to expand its tems
        scroll_to_carousel_and_click(carousel, driver)

        # Set a variable equal to the modal that opens
        reakit_id = reakit_portals[j].find_element(By.CSS_SELECTOR, 'div:nth-child(1) div:nth-child(1)').get_attribute('id')

        modal = driver.find_element(By.CSS_SELECTOR, f'div#{reakit_id} > div:nth-child(2)')

        # Scroll to the bottom of the modal to load all of its elements
        # Careful here: may not load all products. We may have to add slower scrolling
        scroll_to_bottom_of_modal(modal, driver)

        # Get all li objects from the list in the modal. These are the grocery items.
        product_list = modal.find_elements(By.CSS_SELECTOR, 'div:nth-child(1) ul li')

        # Iterate through the listed products
        for product in product_list:
            # Get the image representing the item
            src_value = get_image(product)

            # Get the other textual data about the item
            text_list, unit_price, individual_weight, stock = get_text_data(product)

            # Parse the text data into useful data
            tags, price, name, units, src = parse_text_data(text_list, src_value)

            # Add our dataframe of data to a list of dataframes. These will be concatenated to the full df later.
            cols = df.columns
            row = pd.DataFrame(data=[[original_store, name, price, units, unit_price, individual_weight, tags, stock, src]],
                            columns=cols)

            to_concat.append(row)

        close_button = reakit_portals[j].find_element(By.CSS_SELECTOR, '[aria-label="Close"]')
        close_button.click()

        j += 1

    # Close the browser
    #driver.quit()

    # Add all of our results to the full dataframe
    df = pd.concat([df] + to_concat, ignore_index=True)

    # Filter out missing data
    df = df_filter(df)

    # Save the dataframe to csv
    df.to_csv(f"{'_'.join(original_store.split())}.csv", index=False)

[<selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c2302", element="9B5970F66F3E5DDB79708EE241B99094_element_270")>, <selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c2302", element="9B5970F66F3E5DDB79708EE241B99094_element_271")>, <selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c2302", element="9B5970F66F3E5DDB79708EE241B99094_element_272")>, <selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c2302", element="9B5970F66F3E5DDB79708EE241B99094_element_273")>, <selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c2302", element="9B5970F66F3E5DDB79708EE241B99094_element_274")>, <selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c2302", element="9B5970F66F3E5DDB79708EE241B99094_element_275")>, <selenium.webdriver.remote.webelement.WebElement (session="252453d83024042e29a47191962c

ElementClickInterceptedException: Message: element click intercepted: Element <button disabled="" aria-label="Previous page" class="e-1iv2lwb">...</button> is not clickable at point (1343, 124). Other element would receive the click: <div class="e-herp1y">...</div>
  (Session info: chrome=121.0.6167.85)
Stacktrace:
	GetHandleVerifier [0x00007FF6F0197012+3522402]
	(No symbol) [0x00007FF6EFDB8352]
	(No symbol) [0x00007FF6EFC65ABB]
	(No symbol) [0x00007FF6EFCB3390]
	(No symbol) [0x00007FF6EFCB10D2]
	(No symbol) [0x00007FF6EFCAE834]
	(No symbol) [0x00007FF6EFCAD569]
	(No symbol) [0x00007FF6EFCA1910]
	(No symbol) [0x00007FF6EFCCF05A]
	(No symbol) [0x00007FF6EFCA120A]
	(No symbol) [0x00007FF6EFCCF270]
	(No symbol) [0x00007FF6EFCEBDA3]
	(No symbol) [0x00007FF6EFCCEE03]
	(No symbol) [0x00007FF6EFC9F4D4]
	(No symbol) [0x00007FF6EFCA05F1]
	GetHandleVerifier [0x00007FF6F01C9B9D+3730157]
	GetHandleVerifier [0x00007FF6F021F02D+4079485]
	GetHandleVerifier [0x00007FF6F02175D3+4048163]
	GetHandleVerif

In [ ]:
driver.quit()

In [31]:
to_concat

[]